<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Downloading and Installing pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j4

In [1]:
#@title Importing libraries {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [24]:
#@title Getting SB2021 and FDA database {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

with open(path+'list.txt') as f:
    pdb_codes_list = f.readlines()

pdb_codes_list = [pdb_code[0:4] for pdb_code in pdb_codes_list]

pdb_code = '121P' #@param {type:"string"}


#Ligand list
!rm -rf dbfda.src.txt
source_ligands_file = "http://files.docking.org/catalogs/source/dbfda.src.txt" #@param {type:"string"}
!wget $source_ligands_file


source_ligands = open(r'dbfda.src.txt','r')

ligand_smiles_list = [line.split('\t')[0] for line in source_ligands.readlines()]

source_ligands.close()

In [36]:
#@title Reading molecular files {display-mode: "form"}

#Reading Ligand SMILES
Input = PARSER()

Input.atom_limit = 60 #@param {type:"number"}
    #atom limit not counting H

ligand_smiles = "N[C@@H](CS)C(O)=O" #@param {type:"string"}
print(ligand_smiles)


Lig2 = Mol2()
if Mol2.parse_smiles(Lig2, Input, ligand_smiles, 'Lig2'):
   print('Ligand parsed successfully!')

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')

#Reading reference ligand and receptor from source

lig_src =  path + pdb_code +'/' + pdb_code + '.lig.am1bcc.mol2.gz'
rec_src = path + pdb_code +'/' + pdb_code + '.rec.clean.mol2.gz'

REC = Mol2(Input, rec_src)
RefLig = Mol2(Input, lig_src)



N[C@@H](CS)C(O)=O
Ligand parsed successfully!


In [37]:
print(Lig2.N,REC.N,RefLig.N)

14 2620 46


In [38]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

grid_spacing = 0.5
dim_grid = 30
n_paralell_jobs = 2


scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_3 = (delta_es*delta_es*delta_es)
  Input.deltaij_6 = Input.deltaij_3*Input.deltaij_3

Input.search_box_x, Input.search_box_y, Input.search_box_z = 30.0, 30.0, 30.0
Input.x_dim, Input.y_dim, Input.z_dim = dim_grid, dim_grid, dim_grid
Input.write_grids = True
Input.use_grids = True

Input.grid_spacing = grid_spacing
Input.grid_prefix = "McGrid_"+pdb_code
Input.parallel_jobs = n_paralell_jobs

Input.output = "test_" +pdb_code


In [40]:
#@title Calculating energy {display-mode: "form"}

print(pdb_code)

Writer = WRITER(Input)

Coord = COORD_MC()
HB = FindHB()

HB.find_ligandHB(lig_src,Lig2)

Energy = Energy2(Input)

for k in range(len(list(REC.residue_pointer))-1):
  HB.parse_residue(REC.residue_pointer[k]-1, REC.residue_pointer[k+1]-2, REC.resnames[k], REC, Lig2, 9.0)



com = Coord.compute_com(Lig2)

energy1 = Energy.compute_ene(REC, Lig2, Lig2.xyz)
print("Original energy: %7.3f" % energy1)

start_time = timeit.default_timer()
Grids = Grid(Input, Writer, REC, com)
time = timeit.default_timer() - start_time
print('Time grids: %.2f' %time)

time_per_atom = time/REC.N
print('Time per atom: %.2f'%(time_per_atom))

print('Points: %d %d %d'%(Grids.npointsx,Grids.npointsy,Grids.npointsz))

Writer.write_box(com, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)

energy2 = Energy.compute_ene(Grids, Lig2, Lig2.xyz)
print("Grid Energy: %7.3f" % energy2)

print("Energy error: %7.3f%%" % (100.*(abs(energy2-energy1))/abs(energy1)))



121P
Original energy:  -0.161
Time grids: 54.66
Time per atom: 0.02
Points: 60 60 60
Grid Energy:  -0.162


In [ ]:
#@title Reading Ligand {display-mode: "form"}

Input = PARSER()

Input.atom_limit = 60 #@param {type:"number"}
    #atom limit not counting H

ligand_smiles = "N[C@@H](CS)C(O)=O" #@param {type:"string"}
print(ligand_smiles)


lig = Mol2()
if Mol2.parse_smiles(Lig2, Input, ligand_smiles, 'Lig2'):
  Writer = WRITER(Input)
  !rm -rf Lig2.pdb McLiBELa.pdb
  Writer.write_pdb(Lig2.xyz, Lig2.N, Lig2.atomnames, Lig2.energy, Lig2.rmsd, 'Lig2')
  !gzip -d McLiBELa.pdb.gz
  !mv McLiBELa.pdb Lig2.pdb
  print('Ligand pdb file created successfully!')

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')





N[C@@H](CS)C(O)=O
Ligand pdb file created successfully!


In [ ]:




!sed -i '/END\|CONECT/d'  #tira o end e o conect do arquivo do receptor

!sed -i 's/A  /B  /g'  #troca a cadeia A por B no arquivo do ligante

!sed -i '/CONECT/d'  #tira o conect do arquivo do ligante

complex_pdb = pdb_code + '.pdb'
!cat $rec_pdb  >  #concatena os arquivos

In [9]:
#@title Setting view
%%capture
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol








#convertendo de mol2 pra pdb
rec_pdb = pdb_code+'_rec.pdb'

!obabel -imol2 $rec_src -opdb -O $rec_pdb

lig_pdb = pdb_code+'_lig.pdb'

!obabel -imol2 $lig_src -opdb -O $lig_pdb

!sed -i '/END\|CONECT/d' $rec_pdb #tira o end e o conect do arquivo do receptor

!sed -i 's/A  /B  /g' $lig_pdb #troca a cadeia A por B no arquivo do ligante

!sed -i '/CONECT/d' $lig_pdb #tira o conect do arquivo do ligante

complex_pdb = pdb_code + '.pdb'
!cat $rec_pdb $lig_pdb > $complex_pdb #concatena os arquivos


In [23]:
#@title Docking viewer

view = py3Dmol.view(width=1200,height=800)

#Adding models
view.addModel(open(complex_pdb, 'r').read(),'pdb')


#view.addModel(open('box.pdb', 'r').read(),'pdb')


#Adjusting
#view.setStyle({'chain':"B"},{'stick': {'colorscheme':'yellowCarbon'}})

view.setStyle({'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})

aminoacids = ['MET','THR','ASN','LYS','SER','ARG','VAL','ALA','ASP','GLU','GLY','PHE','LEU','SER','TYR','CYS','TRP','LEU','PRO','HIS','GLN','ARG','ILE','HIE','CYX']

view.setStyle({ 'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})

#view.addStyle({'chain':'A','within':{'distance':'5', 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})




view.setBackgroundColor('white')
view.zoomTo({"model":0})
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Analysing ligands in SMILES format.

try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol


view = py3Dmol.view()

view.addModel(open('Lig2.pdb', 'r').read())

view.setStyle({'model':0},{'stick': {'colorscheme':'cyanCarbon'}})


view.zoomTo({"model":0})
view.show()